<a href="https://colab.research.google.com/github/thiagosierr/PDF-TO-LLM/blob/main/LLM_PDF_to_CHAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain==0.3.0
!pip install langchain-groq==0.2.0
!pip install langchain-community==0.3.0
!pip install pypdf==5.0.0
!pip install langchain sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 27.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.3
    Uninstalling langsmith-0.2.3:
      Successfully uninstalled langsmith-0.2.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 7.5 MB/s eta 0:00:00


# Aula 8: Acessando vídeos de Youtube e PDFs

In [ ]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

api_key = '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'
os.environ['GROQ_API_KEY'] = api_key

chat = ChatGroq(model='llama-3.1-70b-versatile')

path = '/content/74-00(Ignition).pdf'
loader = PyPDFLoader(path)
documents = loader.load()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = FAISS.from_documents(documents, embeddings)


In [ ]:
retriever = vectorstore.as_retriever()

# Criando o template para o prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável que possui as seguintes informações para formular uma resposta: {informacoes}'),
    ('user', '{input}')
])

# Usando o retriever para obter as informações relevantes para a consulta
query = 'QUAL O TORQUE APLICADO NAS VELAS?'
relevant_docs = retriever.get_relevant_documents(query)

# Concatenando as informações relevantes
informacoes_relevantes = "\n".join([doc.page_content for doc in relevant_docs])
chain = prompt | chat


resposta = chain.invoke({'informacoes': informacoes_relevantes, 'input': query})

print(resposta.content)

De acordo com o texto, o torque aplicado nas velas (glow plugs) é de:

* 300 lb.in. (inicialmente)
* Em seguida, soltar para zero torque
* E, finalmente, retighten (apertar novamente) para 300 a 360 lb.in.

Isso está descrito no item C. Operational Check - Glow Plug Ignition System, passo (5).


In [ ]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# API Key do Groq
api_key = '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'
os.environ['GROQ_API_KEY'] = api_key

# Inicializando o modelo ChatGroq
chat = ChatGroq(model='llama-3.1-70b-versatile')

# Carregando o PDF
caminho = '/content/74-00(Ignition).pdf'
loader = PyPDFLoader(caminho)
documents = loader.load()

# Usando embeddings mais potentes
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")  # Modelo mais poderoso
vectorstore = FAISS.from_documents(documents, embeddings)

# Criando o retriever para buscar documentos relevantes, ajustando parâmetros como 'k'
retriever = vectorstore.as_retriever(search_type="similarity", k=2)  # Buscar os 5 documentos mais relevantes

# Criando o template para o prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável que possui as seguintes informações para formular uma resposta: {informacoes}'),
    ('user', '{input}')
])

# Usando o retriever para obter as informações relevantes para a consulta
query = 'Qual página eu encontro o valor de torque das velas?'
relevant_docs = retriever.get_relevant_documents(query)


# Concatenando as informações relevantes
informacoes_relevantes = "\n".join([doc.page_content for doc in relevant_docs])

# Encadeando a cadeia de execução com o prompt e o modelo
chain = prompt | chat

# Obtendo a resposta
resposta = chain.invoke({'informacoes': informacoes_relevantes, 'input': query})

# Imprimindo a resposta
print("\nResposta do modelo:")
print(resposta.content)



Resposta do modelo:
Infelizmente, não há informações sobre o valor de torque das velas nos trechos de texto fornecidos. No entanto, posso sugerir algumas possibilidades:

1. Verifique a seção "IGNITION - MAINTENANCE PRACTICES" (74-00-00), que começa na página 201. É possível que as especificações de torque para as velas sejam mencionadas nessa seção.
2. Procure a seção "Adjustment/Test" (A. General, B. Operational Check - Spark Ignition System, C. Operational Check - Glow Plug Ignition System) na página 201 ou 202. Essas seções podem conter informações sobre como ajustar ou testar as velas, incluindo os valores de torque.
3. Verifique a tabela de conteúdo (TABLE OF CONTENTS) para ver se há alguma seção específica que trate de "Velas" ou "Torque" e procure essa seção.

Se você tiver acesso ao manual completo, é provável que encontre as informações necessárias nas seções mencionadas acima. Se não, pode ser necessário consultar um especialista em manutenção de motores ou um representante

#TESTE